# Imports and Configuration

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
class Args():

  #Select your user name
  user_name = 'Otto'

  imgwidth = 256
  data_linearization = 'serpentina'
  pca_retention = 0.85
  seed = 0
  checkpoint = False
  
  #if data raw and needes to be imported as pkl
  if(user_name == 'Otto'):
    #if data raw is already imported as pkl
    dataFile = "infoShenzhen"+ "_" + data_linearization +".pkl" 
    save_data_path = '/content/gdrive/MyDrive/oneClassSVM/serpentina/'
    save_data_Name = "infoShenzhen"+ data_linearization +".pkl" 
    pathFalse = '/content/gdrive/MyDrive/CXR_png/*_0.png'
    pathTrue = '/content/gdrive/MyDrive/CXR_png/*_1.png'
    save_results_Name = "/content/gdrive/MyDrive/oneClassSVM/serpentina/results/resultsShenzhenSVM.pkl"
    save_results_path = "/content/gdrive/MyDrive/oneClassSVM/serpentina/results/"
    sizePathFalse = 32
    sizePathTrue = 32
    pre_process = "none" #pca,ica,anti-aliasing,data-augmentation
    checkpoint = True
  elif(user_name == 'Regina'):
    #if data raw is already imported as pkl
    dataFile = "Doutorado/Dados_radiografias/infoShenzhen.pkl"
    #pathFalse = 'Imagens_Montgomery/class0/*.png'
    #pathTrue = 'Imagens_Montgomery/class1/*.png'
    save_results_Name = "Doutorado/Resultados/SVM/Shenzhen_85_serp.pkl"
    #sizePathFalse = 32
    #sizePathTrue = 32
    pre_process = "pca"
    checkpoint = False

  n_splits = 10
  


In [ ]:
#imports
import numpy as np
import pandas as pd 
import pickle
import math as mt
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
#import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import PIL
from PIL import Image
from keras.preprocessing.image import img_to_array
import os
import pickle

#from keras.preprocessing.image import array_to_img, img_to_array, load_img

import warnings
warnings.filterwarnings('ignore')
opt = Args()
print(opt)
img_shape = (opt.imgwidth, opt.imgwidth)


# Functions for preprocessing data

In [ ]:
#Data
def image_linearization(img, data_linearization):
  if data_linearization == 'serpentina':
    img = img[:,:,0]
    img = serpentina(img)
  if data_linearization == 'linha':
    img = img[:,:,0]
    img = np.reshape(img,[img.shape[0]*img.shape[1]], order='C')
  if data_linearization == 'coluna':
    img = img[:,:,0]
    img = np.reshape(img,[img.shape[0]*img.shape[1]], order='F')
  return(img)

def serpentina(m_array):
  m_array[1::2] = m_array[1::2, ::-1]
  return m_array.ravel()

def Load_Data( imgwidth, sizePath, file_path, label, data, data_labels, data_linearization, img_names, saveName ):
    
    files = glob.glob(file_path) # your image path
    count = 0
    for myFile in files:
        name = str(myFile)
        name = name[ sizePath:: ]
        img = Image.open(myFile).resize((imgwidth, imgwidth)).convert('RGB')
        img = np.array(img)
        img = image_linearization(img, data_linearization)
        img = img/255 #normalize
        data.append(img)
        data_labels.append(label)
        img_names.append(name)
    return data, data_labels, img_names

def Convert_Data( imgs, data_linearization ):
  data_converted = []
  for img in imgs:
    img = image_linearization(img, data_linearization)
    data_converted.append (img)
    #if data_linearization == 'serpentina':
    #  img = serpentina(img)
    #else:
    #  img = np.reshape(img,[img.shape[0]*img.shape[1]])
  return data_converted

  
def de_img_para_ocsvm(saveName, saveDataPath, pathFalse, pathTrue, sizePathFalse, sizePathTrue, imgwidth, dataLinearization ):
  
  # initializing tb imgs
  data_tb = []
  labels_tb = []
  names_tb = []
  
  # initializing ntb imgs
  data_ntb = []
  labels_ntb = []
  names_ntb = []
  
  #Importando as imagens e transformando em dado apropriado para os modelos na classe False (NTB)
  data_ntb, labels_ntb, names_ntb = Load_Data( imgwidth, sizePathFalse, pathFalse, False , data_ntb, labels_ntb, dataLinearization, names_ntb, saveName )
  #Importando as imagens e transformando em dado apropriado para os modelos na classe True (TB ativa)
  data_tb, labels_tb, names_tb = Load_Data( imgwidth, sizePathTrue, pathTrue, True, data_tb, labels_tb, dataLinearization, names_tb , saveName )

  if not os.path.exists(saveDataPath):
    os.mkdir(saveDataPath)
  os.chdir(saveDataPath)
  infoDataset = { "data": np.concatenate((data_tb, data_ntb)) , "labels": np.concatenate((labels_tb, labels_ntb)), "names": np.concatenate((names_tb, names_ntb)) }
  a_file = open( saveName, "wb" )
  pickle.dump( infoDataset, a_file )
  a_file.close()


# Training 

## Some definitions 

In [ ]:
def split_folds(data, label, names, n_splits, seed, shuffle):

    '''
    Split into different train, validation and test folds
    
    '''
    xTrain, xTest, yTrain, yTest, namesTrain, namesTest = [], [], [], [], [], []

    kf = StratifiedKFold(n_splits=n_splits, random_state=seed, shuffle=shuffle)

    for train_index, test_index in kf.split(data, label):
        xTrain.append( data[ train_index ] )
        xTest.append( data[ test_index ] )
        yTrain.append( label[ train_index ] )
        yTest.append( label[ test_index ] )
        namesTrain.append( names[ train_index ] )
        namesTest.append( names[ test_index ] )
        
    return xTrain, xTest, yTrain, yTest, namesTrain, namesTest

In [ ]:
def sp(classification):
    
    '''
    sp metric
    '''
    sp_i = (classification['1']['recall'] + classification['-1']['recall']) / 2
    sp_i = sp_i * mt.sqrt(classification['1']['recall'] * classification['-1']['recall'])
    sp_i = mt.sqrt(sp_i)
    
    return sp_i

## Class Fold 

In [ ]:
class fold():
    '''
    Fold class, to be easier to perform the calculations
    '''
    
    def __init__(self, 
                 data_train,
                 data_test,
                 label_train,
                 label_test,
                 names_train,
                 names_test,
                 seed,
                 params_grid,
                 shuffle,
                 n_splits,
                 i):
        
        self.xTrain = data_train
        self.xTest = data_test
        self.yTrain = label_train
        self.yTest = label_test
        self.namesTrain = names_train
        self.namesTest = names_test
        self.name = str(i)
        self.params_grid = params_grid
        self.n_splits = n_splits

        #VP, VN, FP and FN from test classification
        self.VP = []
        self.VN = []
        self.FP = []
        self.FN = []
        
        self.sensitivity_mean = 0
        self.specificity_mean = 0
        self.sp_mean = 0
        self.sensitivity_std = 0
        self.specificity_std = 0
        self.sp_std = 0
        
        self.best_params = 0
        
        self.xTrain_folds = []
        self.xCv_folds = []

        self.yTrain_folds = []
        self.yCv_folds = []

        self.namesTrain_folds = []
        self.namesCv_folds = []

        
    def split_folds(self, n_splits, shuffle):
        
        kf = StratifiedKFold(n_splits=n_splits, random_state=seed, shuffle=shuffle)

        for train_index, cv_index in kf.split(self.xTrain, self.yTrain):
            self.xTrain_folds.append( self.xTrain[ train_index ] )
            self.xCv_folds.append( self.xTrain[ cv_index ] )
            self.yTrain_folds.append( self.yTrain[ train_index ] )
            self.yCv_folds.append( self.yTrain[ cv_index ] )
            self.namesTrain_folds.append( self.namesTrain[ train_index ] )
            self.namesCv_folds.append( self.namesTrain[ cv_index ] )
            
    def filter_samples(self, xTrain, yTrain, label):
        
        xTrain = xTrain[yTrain == label]
        
        return xTrain
    
    def gridsearch( self,params_grid, xTrain, xCv, yCv, name, list_params ):
                
        for nu in params_grid['nu']:
            for gamma in params_grid['gamma']:
                for kernel in params_grid['kernel']:
                    #fitting the model
                    model = OneClassSVM(nu=nu, kernel=kernel, gamma=gamma)
                    model.fit(xTrain)
                    
                    #name of the fold + subfold + params
                    name_and_param = name + '_nu_' + str(nu) + '_gamma_' + str(gamma) + '_kernel_' + str(kernel)
                    if opt.checkpoint:
                        model_path = opt.save_results_path + "/models/" + name_and_param + ".pkl"
                        model_file = open( model_path, "wb" )
                        pickle.dump( model, model_file )
                        model_file.close()
                    
                    #original vs predicted
                    yTrue, yPred = yCv, model.predict(xCv)
                    
                    print(name_and_param)
                    #finding the right params object to append
                    for params in list_params:
                        if params.name == 'nu_' + str(nu) + '_gamma_' + str(gamma) + '_kernel_' + str(kernel):
                            classification = classification_report(yTrue, yPred, output_dict=True)
                            params.list_sensitivity.append(classification['1']['recall'])
                            params.list_specificity.append(classification['-1']['recall'])
                            params.list_sp.append(sp(classification))
                    
                    
    def main_subfold(self, j, list_params):
        '''
        main for each subfold
        '''
        #name of the subfold
        name = 'fold_' + self.name + '_subfold_' + str(j)
        print(name)
        
        #filtering
        self.xTrain_folds[j] = self.filter_samples(self.xTrain_folds[j], self.yTrain_folds[j], 1)
        
        #doing the gridsearch for each subfold
        self.gridsearch(self.params_grid, self.xTrain_folds[j], self.xCv_folds[j], self.yCv_folds[j], name, list_params)

    def return_best_sp(self, list_params):
        max = 0
        for params in list_params:
            if params.sp_mean > max:
                max = params.sp_mean
                self.best_params = params

    def classifyVpVnFpFn( self, yTrue, yPred, names ):

      for i in range( len( yTrue ) ):
        if ( yTrue[i] == -1 ): #true value is false
          if ( yPred[i] == -1 ): #right classification in false
            self.VN.append( names[ i ] )
          else: # misclassification in true
            self.FP.append( names[ i ] )
        else: #true value is true
          if ( yPred[i] == 1 ): #right classification in true
            self.VP.append( names[ i ] )
          else: # misclassification in false
            self.FN.append( names[ i ] )
        
    def main(self):
        '''
        main for each fold
        '''
        #split into n folds
        self.split_folds(self.n_splits, shuffle)

        #creating list with params:
        #list with params objects
        list_params = []
        for nu in self.params_grid['nu']:
            for gamma in self.params_grid['gamma']:
                for kernel in self.params_grid['kernel']:
                    #creating params object
                    list_params.append(ParamsOCSVM(nu, gamma, kernel))
        
        #gridsearch in each subfold
        for j in range(len(self.xTrain_folds)):
            self.main_subfold(j, list_params)
            print('terminei subfold :' + str(j))
            
        #mean and std for each param
        for param in list_params:
            param.calculate_mean_std()
            
        #find the best params when it comes to sp
        print('buscando o máximo')
        max = 0
        for param in list_params:
            if param.sp_mean > max:
                max = param.sp_mean
        
        
        self.return_best_sp(list_params)
        print('Best parameters are: ' + self.best_params.name)
        print('SP Index: ' + str(self.best_params.sp_mean) + ' +/- ' + str(self.best_params.sp_std))
        print('Sensitivity Index: ' + str(self.best_params.sensitivity_mean) + ' +/- ' + str(self.best_params.sensitivity_std))
        print('Specitivity Index: ' + str(self.best_params.specificity_mean) + ' +/- ' + str(self.best_params.specificity_std))
        
        return self.best_params

    def fit_and_testing_best_params(self):
        '''
        fitting and testing with the best params
        
        '''
        model = OneClassSVM(nu=self.best_params.nu, gamma=self.best_params.gamma, kernel=self.best_params.kernel)
        xTrain = self.filter_samples(self.xTrain, self.yTrain, 1)
        model.fit(xTrain)
        support_vectors = model.support_vectors_
        
        print('Results best training')
        print('Best parameters are: ' + str(self.best_params.name))
        print('Number of elements in training: ' + str(len(xTrain)))
        print('Number of support vectors: ' + str(len(support_vectors)))
        
        #predicting test data
        yTrue, yPred = self.yTest, model.predict(self.xTest)
        print('Size y and names are: ' + str(len(yTrue)) + ', ' + str(len(self.namesTest)))
        self.classifyVpVnFpFn( yTrue, yPred, self.namesTest )
        classification = classification_report(yTrue, yPred, output_dict=True)
        sensitivity = (classification['1']['recall'])
        specificity = (classification['-1']['recall'])
        sp_ = (sp(classification))
        
        print('Results from testing')
        print('Sensitivity: ' + str(sensitivity))
        print('Specificity: ' + str(specificity))
        print('Sp: ' + str(sp_))

class ParamsOCSVM():
    '''
    Class of the ocsvm params
    '''
    
    def __init__( self, nu, gamma, kernel ):
        
        self.nu = nu
        self.gamma = gamma
        self.kernel = kernel

        
        self.name = 'nu_' + str(self.nu) + '_gamma_' + str(self.gamma) + '_kernel_' + str(self.kernel)
        self.list_sp = []
        self.list_sensitivity = []
        self.list_specificity = []
        
        self.sp_std = 0
        self.sp_mean = 0
        self.sensitivity_std = 0
        self.sensitivity_mean = 0
        self.specificity_std = 0
        self.specificity_mean = 0
        
    def calculate_mean_std(self):
        #mean
        self.sp_mean = np.mean(self.list_sp)
        self.sensitivity_mean = np.mean(self.list_sensitivity)
        self.specificity_mean = np.mean(self.list_specificity)
        
        #std
        self.sp_std = np.std(self.list_sp)
        self.sensitivity_std = np.std(self.list_sensitivity)
        self.specificity_std = np.std(self.list_specificity)

In [ ]:
def filter_samples(X, y, label):
        
        X = X[y == label]
        
        return X


# Reading data

In [ ]:
#if the user want to read the data from raw images
#de_img_para_ocsvm(opt.save_data_Name, opt.save_data_path ,opt.pathFalse, opt.pathTrue, opt.sizePathFalse, opt.sizePathTrue, opt.imgwidth, opt.data_linearization )

In [ ]:
os.chdir(opt.save_data_path)
a_file = open(opt.dataFile, "rb")
info = pickle.load(a_file)

array_labels = np.array( info['labels'] )
array_cxr = np.array( info['data'] )
array_names = np.array( info['names'] )

#Change to one-dimensional array picking first channel
#array_cxr = array_cxr[:,:,:,0]

#Convert to serpentina
#array_cxr = np.array( Convert_Data( array_cxr, 'serpentina' ) )

#changing from false and positive to 0 and 1
array_labels = array_labels.astype(int)

#changing 0 to -1
for z in range(len(array_labels)):
    if array_labels[z] == 0:
        array_labels[z] = -1

# Define grid and split dataset

In [ ]:
params_grid ={    'nu': [0.1,0.2,0.3,0.4,0.5,0.6,0.7, 0.8, 0.9],
                      'gamma': [1.00, 0.01, 0.001, 0.0001, 'scale', 'auto'],
                      'kernel': ['poly', 'rbf', 'sigmoid']}

#creation of the first train and test fold
seed = 0
shuffle = True

xTrain, xTest, yTrain, yTest, namesTrain, namesTest = split_folds(array_cxr, array_labels, array_names, 10, opt.seed, True)

# Training in folds

In [ ]:
list_folds = []
best_params = []
for i in range(opt.n_splits):
    print("Training on fold: " + str(i))
    #Dimensionality reduction with PCA
    if opt.pre_process == "pca":
        pca = PCA( n_components = pca_retention, whiten=True )
        pca.fit( xTrain[i] )
        xTrainPCA = pca.transform( xTrain[i] )  
        xTestPCA = pca.transform( xTest[i] )
        list_folds.append( fold( xTrainPCA[i], xTestPCA[i], yTrain[i], yTest[i], namesTrain[i], namesTest[i], opt.seed, params_grid, shuffle, opt.n_splits, i ) )
        best_params.append( list_folds[i].main() )
    elif opt.pre_process == "none":
        list_folds.append( fold( xTrain[i], xTest[i], yTrain[i], yTest[i], namesTrain[i], namesTest[i], opt.seed, params_grid, shuffle, opt.n_splits, i ) )
        best_params.append( list_folds[i].main() )
    if opt.checkpoint:
        fold_path = opt.save_results_path + "/best_results/fold_and_best_param" + "_"+ str(i) +".pkl"
        foldDataset = { "list_folds": list_folds , "list_best_params": best_params, "fold": list_folds[i] , "best_params": best_params[i]}
        fold_file = open( fold_path, "wb" )
        pickle.dump(foldDataset , fold_file )
        fold_file.close()

    print("Train finished on fold: " + str(i))

Training on fold: 5
fold_5_subfold_0
fold_5_subfold_0_nu_0.1_gamma_1.0_kernel_poly
fold_5_subfold_0_nu_0.1_gamma_1.0_kernel_rbf
fold_5_subfold_0_nu_0.1_gamma_1.0_kernel_sigmoid
fold_5_subfold_0_nu_0.1_gamma_0.01_kernel_poly
fold_5_subfold_0_nu_0.1_gamma_0.01_kernel_rbf
fold_5_subfold_0_nu_0.1_gamma_0.01_kernel_sigmoid
fold_5_subfold_0_nu_0.1_gamma_0.001_kernel_poly
fold_5_subfold_0_nu_0.1_gamma_0.001_kernel_rbf
fold_5_subfold_0_nu_0.1_gamma_0.001_kernel_sigmoid
fold_5_subfold_0_nu_0.1_gamma_0.0001_kernel_poly
fold_5_subfold_0_nu_0.1_gamma_0.0001_kernel_rbf
fold_5_subfold_0_nu_0.1_gamma_0.0001_kernel_sigmoid
fold_5_subfold_0_nu_0.1_gamma_scale_kernel_poly
fold_5_subfold_0_nu_0.1_gamma_scale_kernel_rbf
fold_5_subfold_0_nu_0.1_gamma_scale_kernel_sigmoid
fold_5_subfold_0_nu_0.1_gamma_auto_kernel_poly
fold_5_subfold_0_nu_0.1_gamma_auto_kernel_rbf
fold_5_subfold_0_nu_0.1_gamma_auto_kernel_sigmoid
fold_5_subfold_0_nu_0.2_gamma_1.0_kernel_poly
fold_5_subfold_0_nu_0.2_gamma_1.0_kernel_rbf
fold_

# Apply in test fold and save results

In [ ]:
VP = []
VN = []
FP = []
FN = []

for i in range( n_splits ):
  print( "Calculating for test fold = " + str(i) )
  list_folds[i].fit_and_testing_best_params()
  VP.append( list_folds[i].VP )
  VN.append( list_folds[i].VN )
  FP.append( list_folds[i].FP )
  FN.append( list_folds[i].FN )

#Create dictionary with test results
results = { 'VP': VP, 'VN': VN, 'FP': FP, 'FN': FN }

a_file = open( opt.saveName, "wb" )
pickle.dump( results, a_file )
a_file.close()

Calculating for test fold = 0
Results best training
Best parameters are: nu_0.5_gamma_0.01_kernel_sigmoid
Number of elements in training: 958
Number of support vectors: 494
Size y and names are: 214, 214
Results from testing
Sensitivity: 0.7009345794392523
Specificity: 0.6261682242990654
Sp: 0.6630242503890679
Calculating for test fold = 1
Results best training
Best parameters are: nu_0.5_gamma_0.01_kernel_sigmoid
Number of elements in training: 958
Number of support vectors: 497
Size y and names are: 214, 214
Results from testing
Sensitivity: 0.616822429906542
Specificity: 0.6448598130841121
Sp: 0.6307632254487112
Calculating for test fold = 2
Results best training
Best parameters are: nu_0.5_gamma_0.01_kernel_sigmoid
Number of elements in training: 958
Number of support vectors: 497
Size y and names are: 214, 214
Results from testing
Sensitivity: 0.616822429906542
Specificity: 0.5607476635514018
Sp: 0.5884509841566484
Calculating for test fold = 3
Results best training
Best parameter

In [ ]:
def sensitivity( nVP, nFN ):
  # calculates sensitivity
  # nVP - number of true positives
  # nFN - number of false negatives
  sens = nVP/( nVP + nFN )
  return sens

def specificity( nVN, nFP ):
  # calculates specificity
  # nVN - number of true negatives
  # nFP - number of false positives
  spec = nVN/( nVN + nFP )
  return spec

def sp_index(nVP, nVN, nFP, nFN):
  # calculates sp index
  # nVN - number of true negatives
  # nFP - number of false positives
  # nVP - number of true positives
  # nFN - number of false negatives
  import math

  sens = nVP/( nVP + nFN )
  spec = nVN/( nVN + nFP )
  arit = ( sens + spec )/2
  geom = math.sqrt( sens * spec )
  sp = math.sqrt( arit * geom )
  return sp

sens = []
spec = []
sp = []
for ifold in range( len( VP ) ):
  sens.append( sensitivity( len(VP[ ifold ]), len(FN[ ifold ])) )
  spec.append( specificity( len(VN[ ifold ]), len(FP[ ifold ] ) ))
  sp.append( sp_index( len(VP[ ifold ]), len(VN[ ifold ]), len(FP[ ifold ]), len(FN[ ifold ]) ) ) 

sens_mean = np.mean(sens)
spec_mean = np.mean(spec)
sp_mean = np.mean(sp)
sens_std = np.std(sens)
spec_std = np.std(spec)
sp_std = np.std(sp)
print("Results task " )
print("sensibility: " + str(round(sens_mean,2)) + " +- " + str(round(1.96*sens_std,2)) )
print("specificity: " + str(round(spec_mean,2)) + " +- " + str(round(1.96*spec_std,2)) )
print("sp: " + str(round(sp_mean,2)) + " +- " + str(round(1.96*sp_std,2)) )

Results task 
0.66 +- 0.09
0.68 +- 0.12
0.67 +- 0.07


In [ ]:
best_sensitivity = {}
best_specificity = {}
best_sp = {}
for i in range(n_splits): 
    sensitivity_j = []
    specificity_j = []
    sp_j = []
    for j in range(n_splits):
        model = OneClassSVM(nu=best_params[i].nu, gamma=best_params[i].gamma, kernel=best_params[i].kernel)
        xTrain_aux = filter_samples(xTrain[j], yTrain[j], 1)
        model.fit(xTrain_aux)
        support_vectors = model.support_vectors_
        
        #predicting test data
        yTrue, yPred = yTest[j], model.predict(xTest[j])
        classification = classification_report(yTrue, yPred, output_dict=True)
        sensitivity = (classification['1']['recall'])
        specificity = (classification['-1']['recall'])
        sp_ = (sp(classification))
        
        print('Results from testing')
        print('Sensitivity: ' + str(sensitivity))
        print('Specificity: ' + str(specificity))
        print('Sp: ' + str(sp_))
        sensitivity_j.append(sensitivity)
        specificity_j.append(specificity)
        sp_j.append(sp_)
    params_name = 'nu_' + str(best_params[i].nu) + '_gamma_' + str(best_params[i].gamma) + '_kernel_' + str(best_params[i].kernel)
    best_sensitivity[params_name] = sensitivity_j
    best_specificity[params_name] = specificity_j
    best_sp[params_name] = sp_j
    


Results from testing
Sensitivity: 0.0
Specificity: 1.0
Sp: 0.0
Results from testing
Sensitivity: 0.0
Specificity: 1.0
Sp: 0.0
Results from testing
Sensitivity: 0.0
Specificity: 1.0
Sp: 0.0
Results from testing
Sensitivity: 0.0
Specificity: 1.0
Sp: 0.0
Results from testing
Sensitivity: 0.0
Specificity: 1.0
Sp: 0.0
Results from testing
Sensitivity: 0.0
Specificity: 1.0
Sp: 0.0
Results from testing
Sensitivity: 0.0
Specificity: 1.0
Sp: 0.0
Results from testing
Sensitivity: 0.0
Specificity: 1.0
Sp: 0.0
Results from testing
Sensitivity: 0.0
Specificity: 1.0
Sp: 0.0
Results from testing
Sensitivity: 0.0
Specificity: 1.0
Sp: 0.0
Results from testing
Sensitivity: 0.0
Specificity: 1.0
Sp: 0.0
Results from testing
Sensitivity: 0.0
Specificity: 1.0
Sp: 0.0
Results from testing
Sensitivity: 0.0
Specificity: 1.0
Sp: 0.0
Results from testing
Sensitivity: 0.0
Specificity: 1.0
Sp: 0.0
Results from testing
Sensitivity: 0.0
Specificity: 1.0
Sp: 0.0
Results from testing
Sensitivity: 0.0
Specificity: 1.0


In [ ]:
for i in range(n_splits):
    print(best_params[i].name)

nu_0.5_gamma_0.01_kernel_sigmoid
nu_0.5_gamma_0.01_kernel_sigmoid
nu_0.5_gamma_0.01_kernel_sigmoid
nu_0.5_gamma_scale_kernel_sigmoid
nu_0.5_gamma_0.01_kernel_sigmoid
nu_0.5_gamma_0.01_kernel_sigmoid
nu_0.5_gamma_auto_kernel_sigmoid
nu_0.5_gamma_0.01_kernel_sigmoid
nu_0.5_gamma_scale_kernel_sigmoid
nu_0.5_gamma_auto_kernel_sigmoid


Resultados para Sensibilidade

In [ ]:
avg_Sense = {}
sd_Sense = {}
for k,v in best_sensitivity.items():
    # v is the list of grades for student k
    avg_Sense[k] = np.mean(v)
    sd_Sense[k] = np.std(v)

In [ ]:
print(len(results.VP))

KeyError: ignored

In [ ]:
avg_Sense

{'nu_0.5_gamma_0.01_kernel_sigmoid': 0.0,
 'nu_0.5_gamma_auto_kernel_sigmoid': 0.4977076353376829,
 'nu_0.5_gamma_scale_kernel_sigmoid': 0.500502556868277}

In [ ]:
sd_Sense

{'nu_0.5_gamma_0.01_kernel_sigmoid': 0.0,
 'nu_0.5_gamma_auto_kernel_sigmoid': 0.05390671924983281,
 'nu_0.5_gamma_scale_kernel_sigmoid': 0.04908429895397627}

Resultados para Especificidade

In [ ]:
avg_St = {}
sd_St = {}
for k,v in best_specificity.items():
    # v is the list of grades for student k
    avg_St[k] = np.mean(v)
    sd_St[k] = np.std(v)

In [ ]:
avg_St

{'nu_0.5_gamma_0.01_kernel_sigmoid': 1.0,
 'nu_0.5_gamma_auto_kernel_sigmoid': 0.6831775700934579,
 'nu_0.5_gamma_scale_kernel_sigmoid': 0.6841121495327102}

In [ ]:
sd_St

{'nu_0.5_gamma_0.01_kernel_sigmoid': 0.0,
 'nu_0.5_gamma_auto_kernel_sigmoid': 0.035080980147894326,
 'nu_0.5_gamma_scale_kernel_sigmoid': 0.0417538465489508}

Resultados para o Índice SP

In [ ]:
avg_SP = {}
sd_SP = {}
for k,v in best_sp.items():
    # v is the list of grades for student k
    avg_SP[k] = np.mean(v)
    sd_SP[k] = np.std(v)

In [ ]:
avg_SP

{'nu_0.5_gamma_0.01_kernel_sigmoid': 0.0,
 'nu_0.5_gamma_auto_kernel_sigmoid': 0.5861167211278239,
 'nu_0.5_gamma_scale_kernel_sigmoid': 0.5881160714844993}

In [ ]:
sd_SP

{'nu_0.5_gamma_0.01_kernel_sigmoid': 0.0,
 'nu_0.5_gamma_auto_kernel_sigmoid': 0.02961839900411372,
 'nu_0.5_gamma_scale_kernel_sigmoid': 0.029356386555563406}

Image dimension 256x256

In [ ]:
list_folds = []
best_params = []

for i in range(n_splits):
    list_folds.append(fold(xTrain[i], xTest[i], yTrain[i], yTest[i], seed, params_grid, shuffle, n_splits, i))
    best_params.append(list_folds[i].main())

TypeError: ignored

In [ ]:
list_folds[0].fit_and_testing_best_params()

In [ ]:
list_folds[1].fit_and_testing_best_params()

In [ ]:
list_folds[2].fit_and_testing_best_params()

In [ ]:
list_folds[3].fit_and_testing_best_params()

In [ ]:
list_folds[4].fit_and_testing_best_params()

In [ ]:
sensitivity_j = []
specificity_j = []    
sp_j = []

for j in range(n_splits):
        model = OneClassSVM(nu=0.4, gamma=0.0001, kernel='rbf')
        xTrain_aux = filter_samples(xTrain[j], yTrain[j], 1)
        model.fit(xTrain_aux)
        support_vectors = model.support_vectors_
        
        #predicting test data
        yTrue, yPred = yTrain[j], model.predict(xTrain[j])
        classification = classification_report(yTrue, yPred, output_dict=True)
        sensitivity = (classification['1']['recall'])
        specificity = (classification['-1']['recall'])
        sp_ = (sp(classification))
        
        print('Results from testing')
        print('Sensitivity: ' + str(sensitivity))
        print('Specificity: ' + str(specificity))
        print('Sp: ' + str(sp_))
        sensitivity_j.append(sensitivity)
        specificity_j.append(specificity)
        sp_j.append(sp_)

In [ ]:
best_sensitivity = {}
best_specificity = {}
best_sp = {}
for i in range(n_splits): 
    sensitivity_j = []
    specificity_j = []
    sp_j = []
    for j in range(n_splits):
        model = OneClassSVM(nu=best_params[i].nu, gamma=best_params[i].gamma, kernel=best_params[i].kernel)
        xTrain_aux = filter_samples(xTrain[j], yTrain[j], 1)
        model.fit(xTrain_aux)
        support_vectors = model.support_vectors_
        
        #predicting test data
        yTrue, yPred = yTest[j], model.predict(xTest[j])
        classification = classification_report(yTrue, yPred, output_dict=True)
        sensitivity = (classification['1']['recall'])
        specificity = (classification['-1']['recall'])
        sp_ = (sp(classification))
        
        print('Results from testing')
        print('Sensitivity: ' + str(sensitivity))
        print('Specificity: ' + str(specificity))
        print('Sp: ' + str(sp_))
        sensitivity_j.append(sensitivity)
        specificity_j.append(specificity)
        sp_j.append(sp_)
    params_name = 'nu_' + str(best_params[i].nu) + '_gamma_' + str(best_params[i].gamma) + '_kernel_' + str(best_params[i].kernel)
    best_sensitivity[params_name] = sensitivity_j
    best_specificity[params_name] = specificity_j
    best_sp[params_name] = sp_j

In [ ]:
for i in range(n_splits):
    print(best_params[i].name)

In [ ]:
avg_Sense = {}
sd_Sense = {}
for k,v in best_sensitivity.items():
    # v is the list of grades for student k
    avg_Sense[k] = np.mean(v)
    sd_Sense[k] = np.std(v)

In [ ]:
avg_Sense

In [ ]:
sd_Sense

In [ ]:
avg_St = {}
sd_St = {}
for k,v in best_specificity.items():
    # v is the list of grades for student k
    avg_St[k] = np.mean(v)
    sd_St[k] = np.std(v)

In [ ]:
avg_St

In [ ]:
sd_St

In [ ]:
avg_SP = {}
sd_SP = {}
for k,v in best_sp.items():
    # v is the list of grades for student k
    avg_SP[k] = np.mean(v)
    sd_SP[k] = np.std(v)

In [ ]:
avg_SP

In [ ]:
sd_SP

Cálculo do campeão no conjunto de treinamento

In [ ]:
sensitivityTrain = []
specificityTrain = []    
sp_train = []

for j in range(n_splits):
        model = OneClassSVM(nu=0.4, gamma=0.0001, kernel='rbf')
        xTrain_aux = filter_samples(xTrain[j], yTrain[j], 1)
        model.fit(xTrain_aux)
        support_vectors = model.support_vectors_
        
        #predicting test data
        yTrue, yPred = yTrain[j], model.predict(xTrain[j])
        classification = classification_report(yTrue, yPred, output_dict=True)
        sensitivity = (classification['1']['recall'])
        specificity = (classification['-1']['recall'])
        sp_ = (sp(classification))
        
        print('Results from testing')
        print('Sensitivity: ' + str(sensitivity))
        print('Specificity: ' + str(specificity))
        print('Sp: ' + str(sp_))
        sensitivityTrain.append(sensitivity)
        specificityTrain.append(specificity)
        sp_train.append(sp_)

In [ ]:
print('Sp: ' + str(np.mean(sp_train)) + '+-' + str(np.std(sp_train)))
print('Sensitivity: ' + str(np.mean(sensitivityTrain)) + '+-' +str(np.std(sensitivityTrain)))
print('Specificity: ' +str(np.mean(specificityTrain)) + '+-' + str(np.std(specificityTrain)))